In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, Activation, LSTM
from keras.utils import np_utils

import os

Using TensorFlow backend.


In [2]:
# sample sentence
sentence = "if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea."

In [4]:
sentence

"if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea."

In [5]:
char_set = list(set(sentence))  # id -> char ['i', 'l', 'e', 'o', 'h', ...]
char_dic = {w: i for i, w in enumerate(char_set)}

In [10]:
data_dim = len(char_set)
seq_length = timesteps = 10
nb_classes = len(char_set)

In [11]:
dataX = []
dataY = []
for i in range(0, len(sentence) - seq_length):
    x_str = sentence[i:i + seq_length]
    y_str = sentence[i + 1: i + seq_length + 1]
    print(x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # char to index
    y = [char_dic[c] for c in y_str]  # char to index

    dataX.append(x)
    dataY.append(y)

('if you wan', '->', 'f you want')
('f you want', '->', ' you want ')
(' you want ', '->', 'you want t')
('you want t', '->', 'ou want to')
('ou want to', '->', 'u want to ')
('u want to ', '->', ' want to b')
(' want to b', '->', 'want to bu')
('want to bu', '->', 'ant to bui')
('ant to bui', '->', 'nt to buil')
('nt to buil', '->', 't to build')
('t to build', '->', ' to build ')
(' to build ', '->', 'to build a')
('to build a', '->', 'o build a ')
('o build a ', '->', ' build a s')
(' build a s', '->', 'build a sh')
('build a sh', '->', 'uild a shi')
('uild a shi', '->', 'ild a ship')
('ild a ship', '->', 'ld a ship,')
('ld a ship,', '->', 'd a ship, ')
('d a ship, ', '->', ' a ship, d')
(' a ship, d', '->', 'a ship, do')
('a ship, do', '->', ' ship, don')
(' ship, don', '->', "ship, don'")
("ship, don'", '->', "hip, don't")
("hip, don't", '->', "ip, don't ")
("ip, don't ", '->', "p, don't d")
("p, don't d", '->', ", don't dr")
(", don't dr", '->', " don't dru")
(" don't dru", '->',

In [12]:
# One-hot encoding
dataX = np_utils.to_categorical(dataX, nb_classes=nb_classes)
# reshape X to be [samples, time steps, features]
dataX = np.reshape(dataX, (-1, seq_length, data_dim))
print(dataX.shape)

(170, 10, 25)


In [13]:
# One-hot encoding
dataY = np_utils.to_categorical(dataY, nb_classes=nb_classes)
# time steps
dataY = np.reshape(dataY, (-1, seq_length, data_dim))
print(dataY.shape)

(170, 10, 25)


In [14]:
model = Sequential()
model.add(LSTM(nb_classes, input_shape=(
    timesteps, data_dim), return_sequences=True))
model.add(LSTM(nb_classes, return_sequences=True))
model.add(TimeDistributed(Dense(nb_classes)))

model.add(Activation('softmax'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 10, 25)        5100        lstm_input_1[0][0]               
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 10, 25)        5100        lstm_1[0][0]                     
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (None, 10, 25)        650         lstm_2[0][0]                     
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 10, 25)        0           timedistributed_1[0][0]          
Total params: 10,850
Trainable params: 10,850
Non-trainable params: 0
_____________________

In [17]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop', metrics=['accuracy'])
model.fit(dataX, dataY, nb_epoch=1000)

Epoch 1/1000
170/170 [==============================] - 1s - loss: 2.0286 - acc: 0.4253     
Epoch 2/1000
170/170 [==============================] - 0s - loss: 1.9903 - acc: 0.4382     
Epoch 3/1000
170/170 [==============================] - 0s - loss: 1.9801 - acc: 0.4382     
Epoch 4/1000
170/170 [==============================] - 0s - loss: 1.9673 - acc: 0.4459     

In [18]:
predictions = model.predict(dataX, verbose=0)
for i, prediction in enumerate(predictions):
    # print(prediction)
    x_index = np.argmax(dataX[i], axis=1)
    x_str = [char_set[j] for j in x_index]

    index = np.argmax(prediction, axis=1)
    result = [char_set[j] for j in index]

    print(''.join(x_str), ' -> ', ''.join(result))

('if you wan', ' -> ', 'l you want')
('f you want', ' -> ', 'oyou want ')
(' you want ', ' -> ', 'tou want t')
('you want t', ' -> ', 'ou want to')
('ou want to', ' -> ', '  want to ')
('u want to ', ' -> ', 'pwant to b')
(' want to b', ' -> ', 'tont to bu')
('want to bu', ' -> ', 'ont to bui')
('ant to bui', ' -> ', 'nd to buil')
('nt to buil', ' -> ', 'd to build')
('t to build', ' -> ', 'hao build ')
(' to build ', ' -> ', 'to luild a')
('to build a', ' -> ', 'h luild a ')
('o build a ', ' -> ', ' luild a s')
(' build a s', ' -> ', 'tuild a sh')
('build a sh', ' -> ', 'utld a shi')
('uild a shi', ' -> ', 'pld a ship')
('ild a ship', ' -> ', 'ld a ship,')
('ld a ship,', ' -> ', 'e a ship, ')
('d a ship, ', ' -> ', ' anship, d')
(' a ship, d', ' -> ', 'tnship, do')
('a ship, do', ' -> ', 'nship, don')
(' ship, don', ' -> ', "thip, don'")
("ship, don'", ' -> ', "iip, don't")
("hip, don't", ' -> ', "ep, don't ")
("ip, don't ", ' -> ', "l, don't d")
("p, don't d", ' -> ', "e don't dr")
(